In [ ]:
            ## Make Plots from Climatologies Produced with ObsFlow
### Getting an overview of the catalogue and the various files 

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import xscen as xs
import cartopy.crs as ccrs
import cartopy
from xscen.config import CONFIG
from xclim.core import units
from pathlib import Path

%matplotlib inline
out_dir = '/scen3/braun/data/obs_synthese23/image'
# get the ObsFlow config and project catalog
xs.load_config('/home/braun/python/obsflow/paths_obs.yml', '/home/braun/python/obsflow/config_obs.yml', verbose=(__name__ == "__main__"), reset=True)
pcat = xs.ProjectCatalog('/scen3/braun/data/obs_synthese23/pcat_obs.json')
display(pcat.df)
ds_dict = pcat.search(processing_level='climatology').to_dataset_dict(**CONFIG['to_dataset_dict'])
# remove AHCCD for now
ds_dict = {k: v for k, v in sorted(ds_dict.items()) if 'AHCCD' not in k}

for ds_id, ds_in in ds_dict.items():
    # inspect the dataset
    # print(f'{ds_id}: {ds_in.period.values}')
    # print(f'{ds_id}: {list(ds_in.data_vars.keys())}')
    print(f'{ds_id}: {list(ds_in.dims.keys())}')
    
print('\n\n')

for ds_id, ds_clim in ds_dict.items():
    if 'GovCan_RDRS' in ds_id:
        print(f'{ds_id}:')
        display(ds_dict[ds_id])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15, 6))

#ds_dict['GovCan_RDRS_NAM.Quebec.climatology.QS-DEC'].tg_mean.sel(period='1981-2010', season='JJA').plot(ax=axs[0])
#ds_dict['GovCan_RDRS_NAM.Quebec.climatology.fx'].tas_mean_sea.sel(period='1981-2010', season='JJA').plot(ax=axs[1])
ds_dict['GovCan_RDRS_NAM.Quebec.climatology.MS'].tg_mean.sel(period='1981-2010', month='JAN').plot(ax=axs[0])
ds_dict['GovCan_RDRS_NAM.Quebec.climatology.fx'].tas_mean_mon.sel(period='1981-2010', month='JAN').plot(ax=axs[1])

print(sorted([f'{name}' for name in ds_dict['GovCan_RDRS_NAM.Quebec.climatology.fx'].data_vars]))


### Note: Indicators were computed per year/season/month

In [ ]:
ds = xr.open_dataset('/scen3/braun/data/obs_synthese23/indicators/ECMWF_ERA5-Land_NAM_MS_indicators.zarr', engine='zarr')
ds

### Getting some info about the structure of individual files

In [ ]:
#ds = ds_dict['GovCan_RDRS_NAM.Quebec.climatology.fx']
#ds = ds_dict['ECMWF_ERA5-Land_NAM.Quebec.climatology.fx']
#ds = ds_dict['GovCan_AHCCD_CAN_fx_climatology.zarr']
#ds = ds_dict['MRCC5_ECMWF-ERAint75_OURANOS_CRCM5_historical_NAM-22_bcs.NAM-22.climatology.fx']
#ds = ds_dict['MRCC5_ECMWF-ERAint75_OURANOS_CRCM5_historical_NAM-22_bcs.NAM-22.climatology.QS-DEC']
display(ds)
for ind in ds.data_vars.values():
    print(f'{ind.name}: {ind.dims}')
    sp_dim = [dim for dim in ind.dims if dim not in [v[0] for v in ds.cf.axes.values()] + ['period']]
    print(f'{ind.name}: {sp_dim}')

print([v[0] for v in ds.cf.axes.values()]) #.append('period'))

ds.cf.axes.values()
ds.tas_mean_ann.sel(period='1991-2018').plot(y='lat', x='lon')
#ds.tg_mean.sel(period='1980-1985', season='MAM').plot(y='rlat', x='rlon')


## Making the plots for all stats

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import xscen as xs
import cartopy.crs as ccrs
import cartopy
from xscen.config import CONFIG
from xclim.core import units
from pathlib import Path
import numpy as np

out_dir = '/scen3/braun/data/obs_synthese23/image'
# get the ObsFlow config and project catalog
#xs.load_config('/home/braun/python/obsflow/paths_obs.yml', '/home/braun/python/obsflow/config_obs.yml', verbose=(__name__ == "__main__"), reset=True)
pcat = xs.ProjectCatalog('/scen3/braun/data/obs_synthese23/pcat_obs.json')
# display(pcat.df)
ds_dict = pcat.search(processing_level='climatology').to_dataset_dict(decode_time_delta=False)
# remove AHCCD for now
ds_dict = {k: v for k, v in sorted(ds_dict.items()) if 'AHCCD' not in k}

# Plot the climatologies
for ds_id, ds_in in ds_dict.items():
    # print(f'{ds_id}: {ds_in.period.values}')
    # print(f'{ds_id}: {list(ds_in.data_vars.keys())}')

    # TODO This is just a fix for now
    if 'MRCC' in ds_id:
        ds_in.attrs['source'] = 'CRCM5-ERAint'
    #if 'MRCC' not in ds_id: continue
    #if 'ECMWF_ERA5-Land_NAM.Quebec.climatology.fx' not in ds_id: continue

    # make a plot for each period in the dataset
    print(f'Doing: {ds_id}')
    #display(ds_in)
    for period in ds_in.period.values:
        for ind in ds_in.data_vars.values():
            # skip doubles and non-desired
            skip = ['tas_mean_ann', 'tas_mean_mon', 'tas_mean_sea', 'tasmax_mean_ann', 'tasmax_mean_mon', 'tasmax_mean_sea', 'tasmin_mean_ann', 'tasmin_mean_mon', 'tasmin_mean_sea', 'tas_std_ann', 'tas_std_mon', 'tas_std_sea',  'tasmax_std_ann', 'tasmax_std_mon', 'tasmax_std_sea', 'tasmin_std_ann', 'tasmin_std_mon', 'tasmin_std_sea', 'tn_days_above_20', 'tn_days_above_20_std'] 
            if ind.name not in skip:
                # create the plot ID (maybe done in cleanup?)
                plot_id = f"{ind.attrs['long_name'].lower()} - {ds_in.attrs['source']} ({period})"
                plot_id = plot_id.replace('standard deviation', 'std')
                plot_id = plot_id.replace('30-year ', '').replace('.', '')
                # tg_mean_clim_mean, tg_mean_clim_std, tg_std_clim_mean, tg_std_clim_total
                # replace things in the plot_id 
                labels = {'_mean_clim-mean': '', '_mean_clim-std': 'interannual', '_std_clim-mean': 'intra', '_std_clim-total': 'total'}
                plot_id = plot_id.replace(plot_id.split(' ')[0], f"{[labels[key] for key in labels.keys() if key in ind.name][0]} "
                                                                 f"{[t for t in ['annual', 'seasonal', 'monthly'] if t in plot_id][0]} climate " + plot_id.split(' ')[0]).strip()
                if 'interannual' in plot_id: plot_id = plot_id.replace(' '.join(plot_id.split(' ')[1:3]), '').replace('  ', ' ').strip()
                if 'intra' in plot_id: 
                    id_words = plot_id.split(' ')
                    #plot_id = plot_id.replace(' '.join(id_words[0:6]), f"{' '.join(id_words[1:5])} {' '.join(id_words[0])}-{' '.join(id_words[5:])}").replace('  ', ' ').strip() # I would call this: "climate average of intra-annual/seasonal/monthly std of ..."
                    plot_id = plot_id.replace(' '.join(plot_id.split(' ')[2:6]), '').replace('  ', ' ').strip()
                if 'total' in plot_id: plot_id = plot_id.replace(' '.join(plot_id.split(' ')[3:5]), '').replace('  ', ' ').strip()
                print(f'\tPlotting {ind.name}: {plot_id} ...')
                #display(ind)
                # print(f'{ds_id}: {var} \n\t{ds_in[var].attrs["long_name"]}')
    
                # inspect the variable and determine for plotting:
                # tmp_dim = do we plot one (annual), 4 (seasonal) or 12 (monthly) plots?
                tmp_dim = [dim for dim in ind.dims if dim not in ['lat', 'lon', 'rlat', 'rlon', 'period']] #[v[0] for v in ds.cf.axes.values()] + ['period']]
                if not tmp_dim: tmp_dim = [1]
                # select the data for annual and sort for seasonal
                sel_kwargs = {'year': 'ANN'} if tmp_dim[0] == 'year' else {}
                if tmp_dim[0] == 'season': sel_kwargs = {'season': ['DJF', 'MAM', 'JJA', 'SON']} # setting the order here, too!
                if tmp_dim[0] == 'month': sel_kwargs = {'month': ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP','OCT', 'NOV', 'DEC']}
                # how to arrange subplots
                col = {1: None, 'year': None, 'season': 'season', 'month': 'month'}
                col_wrap = {1: None, 'year': None, 'season': 2, 'month': 3}
                # colors to use
                ticks = None
                if 'std' in ind.name:
                    cmap = 'afmhot_r'
                    cbar_lim = {1: [0, 5], 'year': [0, 5], 'season': [0, 5], 'month': [0, 5]} #list(range(-30, 31, 3))}
                    levels = [0, 0.5, 1, 2, 3, 4, 5, 7.5, 10, 12.5, 15, 20]
                    ticks = levels
                elif any(s in ind.name for s in ['tg', 'tn', 'tx', 'tas']): 
                    cmap = 'RdBu_r'
                    cbar_lim = {1: [-15, 15], 'year': [-15, 15], 'season': [-30, 30], 'month': [-30, 30]} #list(range(-30, 31, 3))}
                    levels = 21
                    ticks = np.linspace(cbar_lim[tmp_dim[0]][0], cbar_lim[tmp_dim[0]][1], levels)[0::2]
                
                # convert from K to degC ToDo: Do in cleanup of workflow!
                if ind.attrs['units'] == 'K' and 'std' not in ind.name:
                    ind = units.convert_units_to(ind, 'degC')
    
                # make the plot
                print(f'\t{tmp_dim[0]} - {col[tmp_dim[0]]} - {col_wrap[tmp_dim[0]]}')
                frame = ind.sel(period=period, **sel_kwargs).plot.contourf(
                    transform=ccrs.PlateCarree(),
                    x='lon', y='lat',
                    col=col[tmp_dim[0]], col_wrap=col_wrap[tmp_dim[0]],
                    subplot_kws={'projection': ccrs.LambertConformal()},
                    cbar_kwargs={'shrink': 0.9, 'ticks': ticks},
                    cmap=cmap, levels=levels,
                    vmin=cbar_lim[tmp_dim[0]][0], vmax=cbar_lim[tmp_dim[0]][1],                                                  
                    add_labels=False,
                );
    
                # get the figure object to add a title, coastlines, configure the axes
                if isinstance(frame, xr.plot.facetgrid.FacetGrid): fig = frame.fig
                if isinstance(frame, cartopy.mpl.contour.GeoContourSet): fig = frame.axes.figure
                 
                
                # decorate
                fig.axes[-1].set_ylabel(f'Temperature ({ind.attrs["units"]})')
                if tmp_dim[0] in {'month', 'season'}:
                    for ax, title in zip(frame.axes.flat, sel_kwargs[tmp_dim[0]]):
                        ax.set_title(title)
                        
                fig.suptitle(plot_id.capitalize(), y=1.03, wrap=True);
    
                for ax in fig.axes:
                    #print(f'{ax} which is a {type(ax)}')
                    if isinstance(ax, cartopy.mpl.geoaxes.GeoAxes):
                        ax.coastlines()
                        ax.margins(0)
                        # TODO This is just a fix for CRCM5-ERAint
                        if 'MRCC' in ds_id:
                            ax.set_extent([-150, -40, 24, 70], crs=ccrs.PlateCarree())
                        else:
                            ax.set_extent([-79, -60, 45, 61], crs=ccrs.PlateCarree())
    
                # save the figure and close it
                out_dir = Path('/scen3/braun/data/obs_synthese23/image') / period
                if not out_dir.exists(): out_dir.mkdir(parents=True, exist_ok=True)
                file_name = plot_id.replace(' ', '_').replace('(', '').replace(')', '').replace('.', '').replace('-_', '')
                print(f'Writing {out_dir}/{file_name}.png')
                fig.savefig(f"{out_dir}/{file_name}.png".replace(' ', '_'), bbox_inches='tight', dpi=300)
                display(fig)
                plt.close(fig)

print('All Done!')

In [ ]:
np.linspace(-30, 30, 21)[0::2]

In [ ]:
plot_id = f"{ind.attrs['long_name'].lower()} - {ds_in.attrs['source']} ({period})"
#plot_id = "30-year_mean_of_monthly_mean_of_daily_maximum_temperature_ERA5-Land_1951-1980".replace('_', ' ')
if '30-year std' in plot_id: plot_id = plot_id.replace('30-year std of ', 'Interannual std of the ')
if '30-year mean' in plot_id: 
    plot_id = plot_id.replace('30-year mean of ', '')
    plot_id = plot_id.replace(plot_id.split(' ')[0], plot_id.split(' ')[0].capitalize() + ' climate')

print(plot_id)
print(plot_id.replace(' ', '_').replace('(', '').replace(')', '').replace('-_', ''))

In [ ]:
import datetime
print([datetime.date(2000, m, 1).strftime('%^b') for m in range(1, 13)])

txt = 'my annual mean of the best things I did with girls in my life'
bla = [t for t in ['annual', 'seasonal', 'monthly'] if t in txt][0]
print('hello, it''s ' + bla +',baby!')


labels = {'_mean_clim-mean': '', '_mean_clim-std': 'interannual std', '_std_clim-mean': 'intra', '_std_clim-total': 'total'}
var = 'tg_mean_clim-std'
print([labels[key] for key in labels.keys() if key in var][0])

In [ ]:
ind.name
